In [70]:
import os
import sys
import subprocess

os.environ["FLAGS_allocator_strategy"] = 'auto_growth'

import cv2
import copy
import numpy as np
import json
import time
import logging
from PIL import Image
import tools.infer.utility as utility
import tools.infer.predict_rec as predict_rec
import tools.infer.predict_det as predict_det
import tools.infer.predict_cls as predict_cls
from tools.infer.predict_system import TextSystem
from ppocr.utils.utility import get_image_file_list, check_and_read_gif
from ppocr.utils.logging import get_logger
from tools.infer.utility import draw_ocr_box_txt, get_rotate_crop_image
import argparse
logger = get_logger()


In [63]:
text_detector = './inference/en_PP-OCRv3_det_infer.onnx'
text_recognizer = './inference/PPOCRv3_rec_lisenceplate_14112022.onnx'
text_classifier = './inference/ch_ppocr_mobile_v2.0_cls_infer.onnx'
use_onnx = True

In [67]:
from paddleocr_config import get_config
args = get_config()

In [68]:
args.image_dir = 'test_imgs/1.png'

In [69]:
image_file = args.image_dir
text_sys = TextSystem(args)


save_results = []
total_time = 0
cpu_mem, gpu_mem, gpu_util = 0, 0, 0
_st = time.time()
count = 0

img = cv2.imread(image_file)
if img is None:
    logger.debug("error in loading image:{}".format(image_file))

starttime = time.time()
dt_boxes, rec_res = text_sys(img)
elapse = time.time() - starttime
total_time += elapse

# for text, score in rec_res:
#     logger.debug("{}, {:.3f}".format(text, score))
print(str(image_file) + "  Predict time of %s: %.3fs" % (image_file, elapse))
res = [{
    "transcription": rec_res[idx][0],
    "points": np.array(dt_boxes[idx]).astype(np.int32).tolist(),
} for idx in range(len(dt_boxes))]
res


test_imgs/1.png  Predict time of test_imgs/1.png: 0.226s


[{'transcription': '67A-106.01',
  'points': [[2, 144], [365, 10], [391, 112], [37, 246]]}]